# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.window import Window
import pyspark.sql.functions as F

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
spark = SparkSession.builder.appName("Quiz").getOrCreate()

# 3) read in the data set located at the path "data/sparkify_log_small.json"
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

# 4) write code to answer the quiz questions 

In [2]:
# Register the DataFrame as a SQL temporary view
user_log.createOrReplaceTempView("user_log_view")

# Question 1

Which page did user id "" (empty string) NOT visit?<br>
<input type='checkbox'> About</input><br>
<input type='checkbox'> Home</input><br>
<input type='checkbox'> Login</input><br>
<input type='checkbox' checked> NextSong</input>

In [3]:
# TODO: write your code to answer question 1
spark.sql("""
SELECT DISTINCT page
FROM user_log_view
WHERE userId = ""
""").show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

Friendly for reusing and applying functions.

# Question 3

How many female users do we have in the data set?<br>
<input type='checkbox' checked> 462</input><br>
<input type='checkbox'> 501</input><br>
<input type='checkbox'> 3820</input><br>
<input type='checkbox'> 5844</input>

In [4]:
# TODO: write your code to answer question 3
spark.sql(
"""
SELECT COUNT(DISTINCT userId) AS `nunique female users`
FROM user_log_view
WHERE gender = "F"
"""
).show()

+--------------------+
|nunique female users|
+--------------------+
|                 462|
+--------------------+



# Question 4

How many songs were played from the most played artist?

In [5]:
# TODO: write your code to answer question 4
spark.sql(
"""
SELECT artist, COUNT(song) AS Artistcount
FROM user_log_view
WHERE page = 'NextSong'
GROUP BY artist
ORDER BY 2 DESC
LIMIT 1
"""
).show()

+--------+-----------+
|  artist|Artistcount|
+--------+-----------+
|Coldplay|         83|
+--------+-----------+



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.<br>
<input type='checkbox'> 5</input><br>
<input type='checkbox' checked> 7</input><br>
<input type='checkbox'> 9</input><br>
<input type='checkbox'> 11</input>

In [6]:
# TODO: write your code to answer question 5
# Step 1: Flag page as homevisit
# Step 2: Cumulate homevisit as period
# Step 3: count period where page = 'NextSong'. We filter page != 'Home' because it is the boundary of each Home and NextSong
# Step 4: Calculate how many songs do users listen to on average between visiting our home page
spark.sql("""
WITH homevisit_tb AS
    (
    SELECT 
        userId, page, ts,
        IF(page = 'Home', 1, 0) AS homevisit
    FROM user_log_view
    WHERE page = 'NextSong'
        OR page = 'Home'
    ),
period_tb AS
    (
    SELECT
        *,
        SUM(homevisit) OVER(PARTITION BY userId ORDER BY ts DESC) AS period
    FROM homevisit_tb
    ),
count_period_tb AS
    (
    SELECT 
        userId, 
        period, 
        COUNT(period) AS count_period
    FROM period_tb
    WHERE page = 'NextSong'
    GROUP BY userId, period
    )
SELECT AVG(count_period)
FROM count_period_tb
""").toPandas()

,avg(count_period)
0,6.898347


In [7]:
# Define udf for spark.sql
spark.udf.register("ishome", lambda page: int(page == 'Home'))

# Step 1: Flag page as homevisit

# Create a new view table
spark.sql("""
SELECT userId, page, ts, ishome(page) AS homevisit
FROM user_log_view
""").createOrReplaceTempView("is_home_table")

In [8]:
# Step 2: Cumulate homevisit as period
cumulative_sum = spark.sql("""
        SELECT 
        *,
        SUM(homevisit) OVER(PARTITION BY userId ORDER BY ts DESC) AS period
    FROM is_home_table
    """)

In [9]:
# Step 3: count period where page = 'NextSong'. We filter page != 'Home' because it is the boundary of each Home and NextSong
# keep the results in a view
cumulative_sum.createOrReplaceTempView("period_table")

# Step 4: Calculate how many songs do users listen to on average between visiting our home page
# find the average count for NextSong
spark.sql("""
WITH count_period AS
    (
    SELECT 
        userId, 
        period, 
        COUNT(period) AS count_period
    FROM period_table
    WHERE page = 'NextSong'
    GROUP BY userId, period
    )
SELECT AVG(count_period)
FROM count_period""").show()

+-----------------+
|avg(count_period)|
+-----------------+
|6.898347107438017|
+-----------------+

